## Made by -----> David Laessker - Peter Fagrell <------



### Perceptron

Since the perceptron is a linnear classifier it can only classify data that is linearly seperable. This means that it can only classify data that can be seperated by a line. We can represent this very arcaheicly by the following:

Trainig data 1 would look like this 

x|x
-|-
x|o

and training data 2 would look like this

o|x
-|-
x|o

The top two boxes are Gothenburg/Sydney bottom two are Paris,  the right boxes are July and the left boxes are December.


As we can see we can easily draw a line that separates the two classes in example 1 but not in example 2. This means that the perceptron can only classify example 1 and not example 2.


### Accuracy 

- PegasosLREG = 0.8309

- PegasosSVC = 0.8431





In [19]:
import numpy as np
from aml_perceptron import LinearClassifier

Our implemetation

In [21]:
class PegasosSVC(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for SVCs.
    """

    def __init__(self, lambda_reg=0.1, n_iter=100000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):

        # Preprocess the data
        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weights
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        # Pegasos algorithm implemented
        # like the peudocode in the paper
        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)

            if y*score <= 1:
                self.w = (1 - n*self.lambda_reg) * self.w + n*y*x
            else:
                self.w = (1 - n*self.lambda_reg) * self.w

class PegasosLREG(LinearClassifier):
    """
    Implementation of the Pegasos algorithm for logistic regression.
    """

    def __init__(self, lambda_reg=0.1, n_iter=100000):
        self.lambda_reg = lambda_reg
        self.n_iter = n_iter

    def fit(self, X, Y):

        # Preprocess the data
        self.find_classes(Y)
        Y_encoded = self.encode_outputs(Y)

        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weights
        n_features = X.shape[1]
        self.w = np.zeros(n_features)
        self.lambda_reg = 1/n_features

        # Pegasos algorithm implemented with logistic regression
        for t in range(1, self.n_iter):
            rand = np.random.randint(0, len(X))
            x, y = X[rand], Y_encoded[rand]

            n = 1/(self.lambda_reg*t)

            score = x.dot(self.w)
            gradient = -y * x * (1 - 1/(1 + np.exp(-y * score)))

            self.w = (1 - n * self.lambda_reg) * self.w - n * gradient
            self.w *= min(1, 1 / (np.sqrt(self.lambda_reg) * np.linalg.norm(self.w)))